<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/advanced_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Q1.a change number of filters

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        

        self.conv1 = nn.Conv2d(3, 6, 5)

        self.conv2 = nn.Conv2d(6, 16, 5)

        # randomly select number of feagtures
        fet1 = np.random.randint(100, 400)

        fet2 = np.random.randint(100, 400)

        
        self.fc1 = nn.Linear(in_features= 16 * 5 * 5, out_features=fet1)  # 6*6 from image dimension
        
        self.fc2 = nn.Linear(in_features=fet1, out_features=fet2)

        self.fc3 = nn.Linear(in_features=fet2, out_features=n_classes)
    
    def forward(self, x):

        x = F.max_pool2d(torch.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(torch.relu(self.conv2(x)), kernel_size=2)
  
        x = x.view(-1, 16 * 5 * 5)
        print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x




In [56]:
import torch
import torchvision
import torchvision.transforms as transforms


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [78]:
import torch.optim as optim

net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [79]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print(outputs.shape, inputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4, 400])
torch.Size([4

KeyboardInterrupt: ignored

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 56 %


I run the net few times and always got between 53-57% of success so i guess the number of features is not very meanninful

### Q1.b change conv2d 

In [87]:
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()
        
        # assuming that nmnist image is 28*28

        stride = 2

        k=5

        pad_H = pad_w = np.floor((28/2-1)*stride-28+k)

        

        pad_H2 = pad_w2 = np.floor((pad_H/2-1)*stride-pad_H+k)

        print(pad_H,pad_w,pad_H2,pad_w2)
        self.conv1 = nn.Conv2d(3, 6, k,padding=(2,2))

        self.conv2 = nn.Conv2d(6, 16, k,padding=(2, 2))


        
        self.fc1 = nn.Linear(in_features= 1024, out_features=120)  
        
        self.fc2 = nn.Linear(in_features=120, out_features=84)

        self.fc3 = nn.Linear(in_features=84, out_features=n_classes)
    
    def forward(self, x):

        x = F.max_pool2d(torch.relu(self.conv1(x)), kernel_size=2)
        x = F.max_pool2d(torch.relu(self.conv2(x)), kernel_size=2)
  
        x = x.view(4, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x




In [88]:
net = LeNet5(len(classes))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

3.0 3.0 3.0 3.0


In [90]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

[1,  2000] loss: 1.914
[1,  4000] loss: 1.680
[1,  6000] loss: 1.542
[1,  8000] loss: 1.461
[1, 10000] loss: 1.380
[1, 12000] loss: 1.330
[2,  2000] loss: 1.253
[2,  4000] loss: 1.224
[2,  6000] loss: 1.209
[2,  8000] loss: 1.179
[2, 10000] loss: 1.175
[2, 12000] loss: 1.162


In [91]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 60 %


this section improved my results to 60%

### Q1.c 